# 集成学习 ensemble learning

集成学习（ensemble learning）可以说是现在非常火爆的机器学习方法了。  
它本身不是一个单独的机器学习算法，而是通过构建并结合多个机器学习器来完成学习任务。  
也就是我们常说的“博采众长”。  

集成学习可以用于
- 分类问题集成，
- 回归问题集成，
- 特征选取集成，
- 异常点检测集成等等，   
可以说所有的机器学习领域都可以看到集成学习的身影。  


<h1 style='color:blue'> Part 1：集成学习框架</h1>

## 1.集成学习概述
从下图，我们可以对集成学习的思想做一个概括。  
对于训练数据集，我们通过若干个个体学习器，通过一定的结合策略，就可以最终形成一个强学习器，以达到博采众长的目的。  

![title](../images/ensemble_001.png)

也就是说，集成学习有两个主要问题需要解决：
- 1. 如何得到若干个个体学习器
- 2. 如何选择一种结合策略，将这些个体学习器集合成一个强学习器。

## 2.集成学习之个体学习器
对于如何得到若干个个体学习器，我们有两种选择。
- 1. 所有的个体学习器都是一个种类的，或者说同质的。   
    - 比如都是决策树个体学习器，或者都是神经网络个体学习器。
- 2. 所有的个体学习不全是一个种类的，或者说异质的。 
    - 比如我们有一个分类问题，对训练集采用支持向量机个体学习器，逻辑回归个体学习器和朴素贝叶斯个体学习器来学习，再通过某种结合策略来确定最终的分类强学习器。
    
目前来说，同质个体学习器的应用是最广泛的，一般我们常说的集成学习的方法都是指的**同质个体学习器**。    
而同质个体学习器使用最多的模型是**CART决策树**和**神经网络**。

**同质个体学习器**按照个体学习器之间是否存在依赖关系可以分为两类：
- 个体学习器之间存在强依赖关系，一系列个体学习器基本都需要串行生成，代表算法是boosting系列算法；
- 个体学习器之间不存在强依赖关系，一系列个体学习器可以并行生成，代表算法是bagging和随机森林（Random Forest）系列算法。

## 3.集成学习之boosting
boosting的算法原理我们可以用一张图做一个概括如下：
![title](../images/ensemble_002.png)

从图中可以看出：   
Boosting算法的工作机制是
1. 首先从训练集用初始权重训练出一个弱学习器1，根据弱学习的学习误差率$e_1$表现来更新训练样本的权重$\alpha_1$，使得之前若学习器1学习误差率高的训练样本点的权重变高，使得这些误差率高的点在后面的弱学习器2中得到更多的重视。
2. 然后基于调整权重后的训练集来训练若学习器2，如此重复进行，直到若学习器数达到事先指定的数目$T$，最终将这个$T$个弱学习器通过集合策略进行整合，得到最终的强学习器。


Boosting系列算法里最著名算法主要有AdaBoost算法和提升树（boosting tree）系列算法。   
提升树系列算法里面应用最广泛的是梯度提升树（Gradient Boosting Tree）。   
AdaBoost和提升树算法的原理在后面会专门讲。

## 4.集成学习之bagging
Bagging的算法原理和boosting不同，它的弱学习器之间没有依赖关系，可以并行生成，我们可以用一张图做一个概括如下：
![title](../images/ensemble_003.png)

从上图可以看出，bagging的个体弱学习器的训练集是通过随机采样得到的。   
通过$T$次的随机采样，我们就可以得到$T$个采样集，对于这$T$个采样集，我们可以分别独立的训练出$T$个弱学习器，再对这$T$弱学习器通过集合策略来得到最终的强学习器。   

对于这里的随机采用有必要做进一步的介绍，这里一般采用的是自助采样法（Boostap sampling），即对于m个样本的原始训练集，我们每次先随机采集一个样本放入采样集，接着把该样本放回，也就是说下次采样时该样本仍有可能被采集到，这样采集m次，最终可以得到m个样本的采样集，由于是随机采样，这样每次的采样集是和原始的训练集不同的，和其他采样集也是不同的，这样得到多个不同的弱学习器。  

随机森林是bagging的一个特化进阶版，所谓的特化是因为随机森林的弱学习器都是决策树。  
所谓的进阶是随机森林在bagging的样本随机采样基础上，又加上了特征的随机选择，其基本思想没有脱离bagging的范畴。  
bagging和随机森林算法的原理会在后面的文章中专门来讲。

## 5.集成学习之结合策略
假定我们得到的T个弱学习器是$\{h_1, h_2,..., h_T\}$。

### 5.1 平均法
对于数值类的回归预测问题，通常使用的集合策略是平均法，也就是说，对于若干个若学习器的输出进行平均得到最终的预测输出。  

最简单的平均是算法平均，也就是说最终预测是  
$$H(x) = \frac{1}{T} \sum_{i=1}^{T} h_i(x)$$

如果每个个体学习器都有一个权重$w$，则最终预测是：
$$H(x) = \sum_{i=1}^T w_i h_i(x)$$
其中，$w_i$是个体学习器$h_i$的权重，通常有
$$ w_i \geq 0, \sum_{i=1}^T = 1$$

### 5.2 投票法
对于分类问题的预测，我们通常使用的是投票法。   
假设我们的预测类别是$\{ c_1, c_2, ... , c_K\}$，对于任意一个预测样本$x$，我们的$T$个弱学习器的预测结果分别是$(h_1(x), h_2(x), h_3(x), ... , h_T(x))$。   

最简单的投票法是相对多数投票法，也就是我们常说的少数服从多数，也就是$T$个弱学习器的对样本$x$的预测结果中，数量最多的类别$c_i$为最终的分类类别。  
如果不止一个类别获得最高票，则随机选择一个做最终类别。  

稍微复杂的投票法是**绝对多数投票法**，也就是我们常说的要票数过半。  
在相对多数投票法的基础上，不光要求获得高票，还要求票过半数，否则会拒绝预测。   

更加复杂的是**加权投票法**，和加权平均法一样，每个弱学习器的分类票数要乘以一个权重，最终将各个类别的加权票数求和，最大的值对应的类别即为最终类别。

### 5.3 学习法
上两节的方法都是对弱学习器的结果做平均或者投票，相比比较简单，但是可能学习误差较大，于是就有了学习法这种方法。  

对于学习法，代表方法是**stacking**，当使用stacking的结合策略时，我们不是对弱学习器的结果做简单的逻辑处理，而是再加上一层学习器，也就是说，我们将训练集弱学习器的学习结果作为输入，将训练集的输出作为输出，重新训练一个学习器来得到最终结果。  

在这种情况下，我们将弱学习器称为初级学习器，将用于结合的学习器称为次级学习器。  

对于测试集，我们首选用初级学习器预测一次，得到次级学习器的输入样本，再用次级学习器预测一次，得到最终的预测结果。  

----

<h1 style='color:blue'> Part 2：Adaboost算法</h1>

集成学习按照个体学习器之间是否存在依赖关系可以分为两类：
1. 个体学习器之间存在强依赖关系--代表算法：boosting系列算法
2. 个体学习器之间不存在强依赖关系。


在boosting系列算法中，**Adaboost**是最著名的算法之一。  
Adaboost既可以用作分类，也可以用作回归。   

## 1. 回顾boosting算法的基本原理
![title](../images/ensemble_002.png)

从图中可以看出，Boosting算法的工作机制是首先从训练集用初始权重训练出一个弱学习器1，根据弱学习的**学习误差率**表现来更新训练样本的权重，使得之前弱学习器1学习误差率高的训练样本点的权重变高，使得这些**误差率高的点**在后面的**弱学习器2**中得到更多的重视。   
然后基于调整权重后的训练集来训练弱学习器2，如此重复进行，直到弱学习器数达到实现指定的数目$T$，最终将这$T$个弱学习器通过集合策略进行整合，得到最终的强学习器。

不过有几个具体的问题Boosting算法没有详细说明：
1. 如何计算学习误差率$e$？
2. 如何得到弱学习权重系数$\alpha$？
3. 如何更新样本权重$D$？
4. 使用何种结合策略？

只要是boosting大家族的算法，都要解决这4个问题。  
那么Adaboost是怎么解决的呢?

## 2. Adaboost算法的基本思路
根据上面的4个问题来看，在Adaboost中是如何解决的。

假设我们的训练样本是
$$T= \{(x_1, y_1), (x_2, y_2), (x_3, y_3), ... , (x_m, y_m)\}$$

训练集的在第$k$个弱学习器的输出权重为
$$ D(k) = (w_{k1}, w_{k2}, ... , w_{km}); w_{1i} = \frac{1}{m}; i=1,2,3,...,m$$

<h2 style='color:red;bold=True'>首先我们看看Adaboost的**分类**问题。<h2>
<h3 style='color:red;bold=True'>学习误差率$e$</h3>

分类问题的误差率很好理解和计算。  
由于多元分类是二元分类的推广，这里假设我们是二元分类问题，输出为$\{-1, 1\}$，则第$k$个弱分类器$G_k(x)$在训练集上的加权误差率为
$$ e_k = P(G_k(x_i) \neq y_i) = \sum_{i=1}^{m} w_{ki} I(G_k(x_i) \neq y_i)$$
其中$I(G_k(x_i) \neq y_i)$代表单个实例的分类是否正确的数量，如果分类正确则为0，分类错误为1。

<h3 style='color:red;bold=True'>权重系数$\alpha$</h3>
接着我们看弱学习器权重系数，对于二元分类问题，第$k$个弱分类器$G_k(x)$的权重系数为
$$\alpha_k = \frac{1}{2} \ln \frac{1-e_k}{e_k}$$

In [16]:
import math
def alpha_k(error):
    f = (1-error)/error
    return 0.5 * math.log(f, math.e)
print(alpha_k(0.5))
print(alpha_k(0.1))

0.0
1.0986122886681098


为什么这样计算弱学习器**权重系数$\alpha$**呢？   

从上式可以看出，如果分类误差率$e_k$越大，则对应的弱分类器权重系数$\alpha_k$越小，也就是说，误差率小的弱分类器权重系数越大。   

具体为什么采用这个权重系数公式，我们在将Adaboost的损失函数优化时再讲。

<h3 style='color:red;bold=True'>样本权重$D$</h3>
第三个问题，更新样本权重$D$。  

假设第$k$个弱分类器的样本集权重系数为$D(k) = (w_{k1}, w_{k2}, ... , w_{km})$，则对应的第$k+1$个弱分类器的样本集权重系数为
$$ w_{k+1, i} = \frac{w_{ki}}{Z_{K}} \exp(- \alpha_k y_i G_k(x_i))$$
等价写法：
$$ w_{k+1, i} =\begin{cases} \frac{w_{ki}}{Z_{K}} e^{-\alpha_k}, G_k(x_i) =y_i
\\ \frac{w_{ki}}{Z_{K}} e^{\alpha_k}, G_k(x_i) \neq y_i
\end{cases}$$
这里$Z_{K}$是规划化因子：
$$Z_k = \sum_{i=1}^m w_{ki} \exp(-\alpha_k y_i G_k(x_i))$$


从$w_{k+1},i$计算公式可以看出，如果第$i$个样本分类错误，则$y_i G_{k}(x_i) < 0$，导致样本的权重在第$k+1$个弱分类中增大，如果分类正确，则权重在第$k+1$个弱分类器中减少。 具体为什么采用样本权重跟新公式，我们在讲Adaboost的损失函数优化时再讲。

In [35]:
def normalize_z(w, ak, y, g):
    '''
    自定义规范化因子Z的计算
    '''
    Z = 0
    for idx, w_i in enumerate(w):
        Z += w_i * math.exp( (-ak) * (y[idx] * g[idx]))
    return Z
w = [0.25, 0.25, 0.25, 0.25]  # 初始化权重
y = [1, 1, 1, 1]   # 输出的y值
g = [1, 1, 1, -1]  # 弱学习器predict输出的y结果
ak = alpha_k(0.25)
Z = normalize_z(w, ak, y, g)
Z

0.8660254037844386

In [37]:
alpha_k(0.25)

0.5493061443340549

In [38]:
def update_weight_D(w, ak, y, g):
    Z = normalize_z(w, ak, y, g)
    w_new = []
    for idx,w_i in enumerate(w):
        w_new.append((w_i / Z) * math.exp(-ak * y[idx] * g[idx]))
    return w_new

w_new = update_weight_D(w, ak, y, g)
w_new
    

[0.16666666666666669,
 0.16666666666666669,
 0.16666666666666669,
 0.5000000000000001]

![title](../images/ensemble_004.png)

####  李航P140的例子


In [55]:
import numpy as np 

x = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
y = [1, 1, 1, -1, -1, -1, 1, 1, 1, -1]
w = [0.1, ] * 10

def G_classifier(v):
    if v > 2.5:
        return -1
    else:
        return 1
    
predict_y = []
for i in x:
    predict_y.append(G_classifier(i))

predict_y = np.array(predict_y)
y = np.array(y)
diff = y-predict_y
wrong_count = 0
for i in diff:
    if i !=0:
        wrong_count += 1
        
error1 = wrong_count / len(x)
alpha_g1 = alpha_k(error1)
alpha_g1

# Z1 = normalize_z(w, alpha_g1, y, predict_y)
w_2 = update_weight_D(w, alpha_g1, y, predict_y)
w_2

[0.07142857142857141,
 0.07142857142857141,
 0.07142857142857141,
 0.07142857142857141,
 0.07142857142857141,
 0.07142857142857141,
 0.16666666666666666,
 0.16666666666666666,
 0.16666666666666666,
 0.07142857142857141]

由于k=1时计算出来的权重系数$\alpha$为0.4236，所以
$$f_1(x) = 0.4236G_1(x)$$

如此重复，得出
$$f_2(x) = 0.4236G_1(x) + 0.6496G_2(x)$$
$$f_3(x) = 0.4236G_1(x) +0.6496G_2(x) + 0.7514G_3(x)$$
于是最终分类器为
$$G(x) = sign[f_3(x)] = sign[0.4236G_1(x) +0.6496G_2(x) + 0.7514G_3(x)]$$

<h3 style='color:red;bold=True'>集合策略</h3>
最后一个问题是集合策略。  
Adaboost分类采用的是加权平均法，
构建基本分类器的线性组合
$$f(x) = \sum_{k=1}^K \alpha_k G_k(x)$$
最终的强分类器为
$$G(x) = sign(f(x))=sign(\sum_{k=1}^K \alpha_k G_k(x))$$

<h2 style='color:red;bold=True'>接着来看Adaboost的回归问题</h2>

由于Adaboost的回归问题有很多变种，这里我们以**Adaboost R2算法**为准。

<h3 style='color:red;bold=True'>学习误差率$e$</h3>
我们先看看回归问题的误差率的问题，对于第k个弱学习器，计算它在训练集上的**最大误差**
$$ E_k = \max|y_i - G_k(x_i)|, i=1,2,3,...,m$$

然后计算每个样本的**相对误差**：
$$e_{ki} = \frac{|y_i - G_k(x_i)|}{E_k}$$

这里是误差损失为线性时的情况:
$$e_{ki} = \frac{|y_i - G_k(x_i)|}{E_k}$$

如果我们用**平方误差**，则
$$e_{ki} =\frac{(y_i - G_k(x_i))^2}{E_k ^2}$$

如果我们用**指数误差**，则
$$e_{ki} =1- \exp (\frac{-y_i +G_k(x_i)}{E_k})$$

最终得到第k个弱学习器的**误差率**:
$$e_k = \sum_{i=1}^{m} w_{ki}e_{ki}$$

<h3 style='color:red;bold=True'>权重系数$\alpha$</h3>
如何得到弱学习器权重系数$\alpha$。
$$\alpha_k = \frac{e_k}{1-e_k}$$

<h3 style='color:red;bold=True'>样本权重$D$</h3>
对于更新样本权重$D$，第$k+1$个弱学习器的样本集权重系数为
$$w_{k+1,i} = \frac{w_{ki}}{Z_k} \alpha ^{1-e^{ki}}$$
这里$Z_k$是规范化因子
$$Z_k = \sum_{i=1}^{m} w_{ki}\alpha^{1-e_{ki}}$$

<h3 style='color:red;bold=True'>结合策略</h3>
最后是结合策略，和分类问题稍有不同，采用的是对加权的弱学习器取中位数的方法，最终的强回归器为
$$f(x) = \sum_{k=1}^{K}(\ln \frac{1}{\alpha_k})g(x)$$
其中，$g(x)$是所有$\alpha_k G_k(x), k=1,2,3,...,K$的中位数。

## 3.Adaboost分类问题的损失函数优化
上面提及了分类Adaboost的弱学习器权重系数公式和样本权重更新公式。  
但是没有解释选择这个公式的原因。  
其实它可以从Adaboost的损失函数推导出来。  

从另一个角度来讲，Adaboost的模型是**加法模型**，学习算法为**前向分布学习算法**，损失函数为**指数函数的分类问题**。


### 3.1 Adaboost的模型是加法模型
Adaboost的模型是加法模型很好理解，我们的最终的强分类器是若干个弱分类器加权平均而得到的。

### 3.2 前向分布学习算法
前向分布学习算法也好理解，我们的算法是通过一轮轮的弱学习器学习，利用前一个弱学习器的结果更新后一个弱学习器的训练权重。  

也就是说，第$k-1$轮的强学习器为
$$f_{k-1}(x) = \sum_{i=1}^{k-1} \alpha_{i}G_i(x)$$

而第$k$轮强学习器为
$$f_k(x) = \sum_{i=1}^k \alpha_i G_i(x)$$

上两式一比较可以得到
$$f_k(x) = f_{k-1}(x) + \alpha_k G_k(x)$$
可见强学习器的确是通过前向分布学习算法一步步而得到的。


### 3.3 损失函数为指数函数的分类问题
Adaboost损失函数为指数函数，即定义损失函数为
$$\underbrace {\arg \min}_{\alpha, G}\sum_{i=1}^m exp(-y_i f_k(x))$$

李航书中：
AdaBoost最基本的性质是它能在学习过程中不断减少训练误差，即在训练数据集上的分类误差率。  

关于这个问题有下面的定理：
**定理：AdaBoost的训练误差界**
Adaboost算法最终分类器的训练误差界为
$$\frac{1}{N} \sum_{i=1}^N I(G(x_i)\neq y_i) \leq \frac{1}{N} \sum_{i=1} \exp(-y_i f(x_i)) = \prod_m Z_m$$
其中，$$G(x) = sign(f(x))=sign(\sum_{k=1}^K \alpha_k G_k(x))$$
$$f(x) = \sum_{k=1}^K \alpha_k G_k(x)$$
这里$Z_{K}$是规划化因子：
$$Z_k = \sum_{i=1}^m w_{ki} \exp(-\alpha_k y_i G_k(x_i))$$



利用前向分布学习算法的关系可以得到损失函数为：
$$(\alpha_k, G_k(x)) = \underbrace{\arg \min}_{\alpha,G} \sum_{i=1}^{m} \exp[(-y_i)(f_{k-1}(x) + \alpha G(x))]$$

令$w'_{ki} = \exp(-y_i f_{k-1}(x))$，它的值不依赖于$\alpha，G$，因此与最小化无关，仅仅依赖于$f_{k-1}(x)$，随着每一轮迭代而改变。  

将这个式子代入损失函数，损失函数转化为
$$(\alpha_k, G_k(x)) = \underbrace{\arg \min}_{\alpha,G} \sum_{i=1}^{m}w'_{ki} \exp[ \alpha G(x))]$$

首先，我们求$G_k(x)$，可以得到
$$G_k(x) = \underbrace{\arg \min}_{G} \sum_{i=1}^m w'_{ki} I(y_i \neq G(x_i))$$

将$G_k(x)$代入损失函数，并对$\alpha$求导，使其等于0，则得到
$$\alpha_k = \frac{1}{2} \ln \frac{1-e_k}{e_k}$$
其中，$e_k$即为我们前面的分类误差率。  

$$e_k = \frac{\sum_{i=1}^m w'_{ki} I(y_i \neq G(x_i))}{\sum_{i=1}^{m}w'_{ki}} =\sum_{i=1}^m w_{ki}I(y_i \neq G(x_i))$$

最后看样本权重的更新。  
利用$f_k(x) = f_{k-1}(x) + \alpha_{k} G_{k}(x)$和$w'_{ki} = \exp(-y_i f_{k-1}(x))$，即可得：
$$w'_{k+1},i = w'_{ki}\exp(-y_i \alpha_k G_k(x)$$
这样我们就得到了上面讲的分类的Adaboost算法中的样本权重更新公式。

## 4. AdaBoost二元分类问题算法流程
这里我们对AdaBoost二元分类问题算法流程做一个总结。
<a href='http://www.cnblogs.com/pinard/p/6133937.html'> 汇总的流程</a>

输入：样本集$T = \{(x_1, y_1), (x_2, y_2), (x_3, y_3), ... , (x_m, y_m)\}$，输出为$\{-1, +1\}$   
弱分类器算法：迭代次数K。

输出为：最终的强分类器$G(x)$

对于Adaboost多元分类算法，其实原理和二元分类类似，最主要的区别在弱分类器的系数上。  

比如：Adaboost SAMME算法，它的弱分类器的系数为
$$\alpha_k = \frac{1}{2} \ln \frac{1-e_k}{e_k} + \ln (R-1)$$
其中R为类别数。  
如果是二元分类，R=2，则上式和我们的二元分类算法中的弱分类器的系数一致。

## 5. AdaBoost回归问题算法流程

<a href='http://www.cnblogs.com/pinard/p/6133937.html'> 汇总的流程</a>



## 6. AdaBoost算法的正则化
为了防止AdaBoost过拟合，我们通常也会加入正则化项，这个正则化项我们通常称为**步长learning rate**。 

**步长 Learning Rate**定义为$v$。

对于前面的弱学习器的迭代：
$$f_k(x) = f_{k-1}(x) + \alpha_k G_k(x)$$
如果我们加上了正则化项，则有
$$f_k(x) = f_{k-1}(x) + v \alpha_k G_k(x)$$

$v$的取值范围为$0 \leq v \leq 1$。 
对于同样的训练集学习效果，较小的$v$意味着我们需要更多的弱学习器的迭代次数。 通常我们用步长和迭代最大次数一起来决定算法的拟合效果。

## 7. AdaBoost小结
前面没有提到AdaBoost的弱学习器的类型。  

理论上任何学习器都可以用于AdaBoost。  
但一般来说，使用最广泛的AdaBoost弱学习器是决策树和神经网络。  

对于决策树，AdaBoost分类用于CART分类树，而Adaboost回归用了CART回归树。  
AdaBoost算法的优缺点：
优点：
- AdaBoost作为分类器时，分类精度很高
- 在AdaBoost的框架下，可以使用各种回归分类模型来构建弱学习器，非常灵活
- 作为简单的二元分类器时，构造简单，结果可理解。
- 不容易发生过拟合

缺点：
- 对异常样本敏感，异常样本在迭代中可能会获得较高的权重，影响最终的强学习器的预测准确性。